# OWL-DEV: Send to Cloud InfluxDB 

In [ ]:
import mflib 
print(f"mflib version  {mflib.__version__} " )
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
from mflib import owl as owl

#  Using local copy of owl.py for testing
#import owl as owl                                                   

In [ ]:
try:
    fm = fablib_manager()
                     
    fm.show_config()
except Exception as e:
    print(f"Exception: {e}")

## Get slice and nodes

In [ ]:
fm.list_slices()

In [ ]:
%%time

slice_name = "test_slice"

try:
    slice = fm.get_slice(name=slice_name)
except Exception as e:
    print(f"Fail: {e}")
print(slice)

In [ ]:
nodes = slice.get_nodes()

## Enter the docker image name

Image will be pulled from Docker Hub by default.
If needing bo build from a Dockerfile, use [this](./OWL-DEV_build_image_from_Dockerfile.ipynb) in the same directory and change the image name. (e.g., "owl-test:latest")

In [ ]:
# if specific tag needed, add it like: "fabrictestbed/owl:v0.1.2"
image_name="fabrictestbed/owl:v0.2.0"  

# If built from Dockerfile for testing 
#image_name="owl-test:latest"  

#### (Optional) Pull Docker image on each exp node

Docker start (in the cells below) should pull the image automatically.
But if pulling the image is necessary, use these lines.

In [ ]:
# for node in nodes:
#     print(node.get_name())
#     node.execute(f"sudo docker pull {image_name}") 

## Send PCAP Data From Experimental Node(s) to InfluxDB 

OWL stores packet capture data by storing `tcpdump` output to a file with the format: `<IP_address>.pcap`. 

`<IP_address>` is the address of the node that the capture file resides in.

Below, you need to specify value of `influxdb_sender_node_name` to one of the Experimental Node names. OWL will then send packet capture data from this node to the Measurement Node.

In the following cell, the authentication information for InfluxDB must have previously been set up. **Make sure you have run that [cell](#influx_authentication) and return here.**

In [ ]:
# for node in nodes:
#     node.execute('docker remove owl-to-influx')

In [ ]:
influxdb_connection = {"influxdb_url":"", "influxdb_token":"", "influxdb_org":"",
                       "influxdb_bucket":"",  }

influxdb_connection["influxdb_token"] = "<TOKEN>"
influxdb_connection["influxdb_org"] = "RC"
influxdb_connection["influxdb_url"] = "https://us-east-1-1.aws.cloud2.influxdata.com"
influxdb_connection["influxdb_bucket"] = "owl-test1"

In [ ]:
for node in slice.get_nodes():
    influxdb_sender_node_name = node.get_name()
    pcap_file_name = owl.get_node_ip_addr(slice, influxdb_sender_node_name) + ".pcap"   # IP address of node from which to send to InfluxDB
    print(f"Planning to send stored .pcap data from Node: {influxdb_sender_node_name}")
    print(f"pcap Filename = {pcap_file_name}")
    slice = fm.get_slice(slice_name)
    _desttype = "cloud"
    owl.send_to_influxdb(desttype=_desttype, node=slice.get_node(name=influxdb_sender_node_name), 
                         pcapfile=pcap_file_name,
                         img_name=image_name,
                         influxdb_token=influxdb_connection["influxdb_token"],
                         influxdb_org=influxdb_connection["influxdb_org"], 
                         influxdb_url=influxdb_connection["influxdb_url"],
                         influxdb_bucket=influxdb_connection["influxdb_bucket"])

## Check the status of containers

In [ ]:
owl.check_owl_all(slice)

## Stop InfluxDB container on one node

In [ ]:
node_name = "node1"

owl.stop_influxdb_sender(slice, slice.get_node(name=node_name))

## Stop all of them

In [ ]:
owl.stop_owl_all(slice)

## (For debugging) Download a pcap file

In [ ]:
node0 = slice.get_node(name="node0")
pcap_file = '10.136.7.2.pcap'

In [ ]:
node0.download_file(f"/home/fabric/work/my_notebooks/Latency_monitoring/ConfirmTesting/{pcap_file}", f"/home/rocky/owl-output/{pcap_file}")